In [1]:
!pip install pyclipper

     |████████████████████████████████| 133kB 2.9MB/s 


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import shutil, os
os.chdir('/content')
directory = '/content/cloned-repo'
if os.path.exists(directory):
  shutil.rmtree(directory)
else:
  print('Directory not found, creating new directory...')
  !git clone https://github.com/kavyajeetbora/PSENet.pytorch.git /content/cloned-repo
  os.chdir('/content/cloned-repo')
  !ls

Directory not found, creating new directory...
Cloning into '/content/cloned-repo'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 368 (delta 38), reused 15 (delta 7), pack-reused 296
Receiving objects: 100% (368/368), 8.13 MiB | 5.56 MiB/s, done.
Resolving deltas: 100% (164/164), done.


In [18]:
## unzipping the files
from zipfile import ZipFile

def unzip_files(file,output_dir):
  with ZipFile(file, 'r') as zipObj:
    # Extract all the contents of zip file in current directory
    zipObj.extractall(output_dir)
  print('Extracted',len(output_dir),'to output dir')

image_files = '/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/AI4Bharat Dataset/Training Set/Images/Images.Zip'
annotation_files = '/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/AI4Bharat Dataset/Training Set/Annotations/Annotations.Zip'

directory = '/content/Train Set'
if os.path.isdir(directory):
  shutil.rmtree(directory)
  print('removed directory')
  os.mkdir('Train Set')
else:
  print('Directory not found, creating new directory...')
  os.mkdir('Train Set')

Directory not found, creating new directory...


In [19]:
%%time
unzip_files(image_files, output_dir='Train Set/Images')

Extracted 16 to output dir
CPU times: user 1.22 s, sys: 429 ms, total: 1.65 s
Wall time: 1.71 s


In [20]:
%%time
unzip_files(annotation_files, output_dir='Train Set/Annotations')

Extracted 21 to output dir
CPU times: user 504 ms, sys: 257 ms, total: 762 ms
Wall time: 777 ms


In [21]:
print(len(os.listdir('Train Set/Images')))

5000


In [22]:
print(len(os.listdir('Train Set/Annotations')))

5000


In [0]:
from dataset.data_utils import *
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset

In [25]:
data_dir  = 'Train Set'
len(os.listdir(data_dir+ '/Images'))

5000

In [26]:
os.listdir(data_dir)

['Annotations', 'Images']

In [28]:
train_data = MyDataset('Train Set',transform=transforms.ToTensor())
len(train_data)

5000

In [0]:
device = torch.device("cuda:0")

In [41]:
%%time
train_loader = DataLoader(dataset=train_data, batch_size=4, shuffle=True)
img, labels, training_mask = next(iter(train_loader))
print(img.size(), labels.size(), training_mask.size())

torch.Size([4, 3, 640, 640]) torch.Size([4, 6, 640, 640]) torch.Size([4, 640, 640])
CPU times: user 107 ms, sys: 62.8 ms, total: 170 ms
Wall time: 163 ms


In [0]:
img, labels, training_mask = img.to(device), labels.to(device), training_mask.to(device)

In [0]:
from models import PSENet
from models.loss import PSELoss
import torch

model = PSENet(backbone='resnet18', pretrained=True, result_num=6, scale=0.5)
model.to(device)
criterion = PSELoss(Lambda=0.7, ratio=3, reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr=2e-3)

In [45]:
y1 = model(img)
loss_c, loss_s, loss = criterion(y1, labels, training_mask)

print(y1.size())
print(loss_c, loss_s, loss)

torch.Size([4, 6, 640, 640])
tensor(1., device='cuda:0', grad_fn=<MeanBackward0>) tensor(1., device='cuda:0', grad_fn=<MeanBackward0>) tensor(1., device='cuda:0', grad_fn=<AddBackward0>)


## Training

In [0]:
!chmod +x install_dependencies.sh # make shell script executable
!./install_dependencies.sh # run the shell script

chmod: cannot access 'install_dependencies.sh': No such file or directory
/bin/bash: ./install_dependencies.sh: No such file or directory
